In [1]:
import pandas as pd
import numpy as np

import random
from datetime import datetime, timedelta


In [2]:
def minute2hours(mins):
    return int(np.round(mins / 60, 0))

duration_lookup = {
    ("LHR", "CDG"): minute2hours(80),
    ("LHR", "AMS"): minute2hours(80),
    ("CDG", "FRA"): minute2hours(85),
    ("BCN", "MXP"): minute2hours(105),
    ("DUB", "CPH"): minute2hours(140),
}


In [3]:
# LHR - London, Heathrow
# CDG - Paris, Charles De Gaulle
# AMS - Amsterdam, Schiphol
# FRA - Frankfurt
# BCN - Barcelona, El Prat 
# MXP - Milan, Malpensa
# DUB - Dublin
# CPH - Copenhagen, Kastrup
# PRG - Prague, Vaclav Havel
# ZUR - Zurich, Kloten
# FAO - Faro
# LIS - Lisbon
# KEF - Reykjavik, Keflavik
# ROM - Rome, Fiumicino

# Sample airport and aircraft pools
airports = ["LHR", "CDG", "AMS", "FRA", "BCN", "MXP", "DUB", "CPH", "PRG", "ZUR", "FAO", "LIS", "KEF", "ROM"]
aircraft_types = ["A320", "A319", "B737"]

In [ ]:


def generate_flight_schedule(days=5, flights_per_day=50):
    # Define parameters
    airports = ["LHR", "CDG", "AMS", "FRA", "BCN", "MXP", "DUB", "CPH", "PRG", "ZUR", "FAO", "LIS", "KEF", "ROM"]
    crew_bases = ['KEF', 'BCN', 'BCN', 'LHR', 'ZUR']
    aircraft_types = ["A320", "A319", "B737"]
    #aircraft_ids = [f"A320-{i}" for i in range(1, 11)]
    time_blocks = {
        'morning': (6, 9),
        'midday': (11, 14),
        'evening': (17, 21)
    }

    schedule = []

    for day in range(1, days + 1):
        used_flight_numbers = set()
        for _ in range(flights_per_day):
            # Generate unique flight number
            while True:
                flight_number = f"FR{random.randint(100, 999)}"
                if flight_number not in used_flight_numbers:
                    used_flight_numbers.add(flight_number)
                    break

            # Select origin and destination
            origin = random.choice(airports)
            destination = random.choice([a for a in airports if a != origin])

            # Select time block and generate realistic departure time
            block = random.choice(list(time_blocks.values()))
            dep_hour = random.randint(block[0], block[1] - 1)
            dep_minute = random.choice([0, 15, 30, 45])
            dep_time = datetime(2025, 1, 1, dep_hour, dep_minute)

            # Flight duration: 1–3 hours
            duration = timedelta(minutes=random.randint(60, 180))
            arr_time = dep_time + duration

            # Format times
            dep_str = dep_time.strftime("%H:%M")
            arr_str = arr_time.strftime("%H:%M")

            # Assign aircraft and crew base
            aircraft = random.choice(aircraft_types)
            crew_base = random.choice([b for b in crew_bases if b == origin] or crew_bases)

            schedule.append({
                "day": f"{day}",
                "id": flight_number,
                "origin": origin,
                "dest": destination,
                "dep": dep_str,
                "arr": arr_str,
                "type": aircraft,
                #"Crew Base": crew_base
            })

    return pd.DataFrame(schedule)

# Example usage
flights_df = generate_flight_schedule()
flights_df


,day,id,origin,dest,dep,arrival,type
0,1,FR733,FRA,CPH,17:00,18:37,B737
1,1,FR524,PRG,LIS,12:15,14:36,A319
2,1,FR942,PRG,DUB,12:15,14:40,A319
3,1,FR633,MXP,AMS,20:45,23:37,A320
4,1,FR548,LHR,CPH,07:45,10:38,A319
...,...,...,...,...,...,...,...
245,5,FR541,KEF,CPH,12:15,14:33,A319
246,5,FR745,MXP,LHR,18:15,19:29,A319
247,5,FR899,MXP,LIS,11:00,13:29,B737
248,5,FR892,LHR,AMS,11:15,13:06,A319


In [ ]:
# random seed for reproducibility
np.random.seed(1223)

# Generate 250 flights
flights = []
for day in range(1, 6):  # Day 1 to Day 5
    for i in range(50):
        fid = f"D{day}_F{i:03d}"
        origin, dest = np.random.choice(airports, 2, replace=False)
        dep_hour = np.random.randint(6, 20)
        duration = duration_lookup.get((origin, dest), np.random.randint(1, 3))
        arr_hour = dep_hour + duration
        aircraft = np.random.choice(aircraft_types)

        flights.append({
            "day": day,
            "id": fid,
            "origin": origin,
            "dest": dest,
            "dep": f"{dep_hour:02d}:00",
            "arr": f"{arr_hour:02d}:00",
            "duration": duration,
            "type": aircraft
        })



flights_df = pd.DataFrame(flights)

flights_df['dep'] = pd.to_datetime(flights_df['dep'], format='%H:%M').dt.time
flights_df['arr'] = pd.to_datetime(flights_df['arr'], format='%H:%M').dt.time
flights_df = flights_df.sort_values(by=['day', 'dep', 'arr'])
flights_df = flights_df.reset_index(drop=True)

flights_df

In [7]:
flights_df.to_json('data/flights.json')